# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

#from l2hmc_eager import gauge_dynamics_eager as gde

In [2]:
#tf.enable_eager_execution()
tfe = tf.contrib.eager

#from u1_model_eager import GaugeModelEager, train_one_iter
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

Using TensorFlow backend.


In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [4]:
params = {
    'time_size': 12,
    'space_size': 12,
    'link_type': 'U1',
    'dim': 2,
    'beta': 1.,
    'beta_init': 1.,
    'beta_final': 8.,
    'num_samples': 5,
    'num_steps': 5,
    'eps': 0.2,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 5000,
    'save_steps': 1000,
    'logging_steps': 50,
    'annealing_steps': 50,
    'annealing_factor': 0.97,
    'clip_value': 10.,
    'rand': False,
    'metric': 'l2',
    'training_samples_steps': 500,
    'training_samples_length': 100
}

## Create model

In [5]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [6]:
model = GaugeModel(params=params, 
                   config=config,
                   sess=None,
                   conv_net=True,
                   hmc=False,
                   log_dir=None,
                   restore=False,
                   eps_trainable=True,
                   aux=True,
                   annealing=True,
                   clip_grads=False)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_146/
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_146/samples_history.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_146/samples_history/training.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_146/train_samples.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_146/annealing_samples.
log_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_146/
info_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_146/run_info/
figs_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_146/figures/
conv_net: True
hmc: False
aux: True
eps_trainable: True
clip_grads: False
annealing: True
beta: 1.0
time_size: 12
space_size: 12
link_type: U1
dim: 2
beta_init: 1.0
beta_final: 8.0
num_samples: 5
num_steps: 5
eps: 0.2
loss_scale: 0.1
loss_eps: 0.0001
learning_rate_init: 0.0001
learning_rate_decay_steps: 100
learning_rate_decay_rate: 0.98
train_steps: 

In [ ]:
model.train(model.train_steps, kill_sess=False)

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------


In [25]:
run_steps = [100, 200, 500, 1000, 5000]#, 10000]

In [26]:
model.beta_final

8.0

In [27]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...
step:      0/100    	 time/step (normalized): 0.0002521
step:     10/100    	 time/step (normalized): 3.193e-05
step:     20/100    	 time/step (normalized): 4.396e-05
step:     30/100    	 time/step (normalized): 3.91e-05
step:     40/100    	 time/step (normalized): 3.684e-05
step:     50/100    	 time/step (normalized): 3.923e-05
step:     60/100    	 time/step (normalized): 3.932e-05
step:     70/100    	 time/step (normalized): 3.797e-05
step:     80/100    	 time/step (normalized): 4.185e-05
step:     90/100    	 time/step (normalized): 4.247e-05
done.
Samples saved to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_144/samples_history/samples_history_100.pkl.
--------------------------------------------------------------------------------

Running (trained) L2HMC sampler for 200 steps...
step:      0/200    	 time/step (normalized): 3.727e-05
step:     10/200    	 time/step (normalized): 3.723e-05
step:     20/200    	 time/step (

step:    530/1000   	 time/step (normalized): 4.178e-05
step:    540/1000   	 time/step (normalized): 3.8e-05
step:    550/1000   	 time/step (normalized): 3.898e-05
step:    560/1000   	 time/step (normalized): 3.897e-05
step:    570/1000   	 time/step (normalized): 3.985e-05
step:    580/1000   	 time/step (normalized): 3.783e-05
step:    590/1000   	 time/step (normalized): 4.058e-05
step:    600/1000   	 time/step (normalized): 3.724e-05
step:    610/1000   	 time/step (normalized): 3.523e-05
step:    620/1000   	 time/step (normalized): 3.53e-05
step:    630/1000   	 time/step (normalized): 3.268e-05
step:    640/1000   	 time/step (normalized): 3.695e-05
step:    650/1000   	 time/step (normalized): 3.594e-05
step:    660/1000   	 time/step (normalized): 4.151e-05
step:    670/1000   	 time/step (normalized): 4.112e-05
step:    680/1000   	 time/step (normalized): 3.618e-05
step:    690/1000   	 time/step (normalized): 3.598e-05
step:    700/1000   	 time/step (normalized): 4.808

step:    960/5000   	 time/step (normalized): 3.316e-05
step:    970/5000   	 time/step (normalized): 3.203e-05
step:    980/5000   	 time/step (normalized): 3.241e-05
step:    990/5000   	 time/step (normalized): 3.243e-05
step:   1000/5000   	 time/step (normalized): 3.172e-05
step:   1010/5000   	 time/step (normalized): 3.23e-05
step:   1020/5000   	 time/step (normalized): 3.319e-05
step:   1030/5000   	 time/step (normalized): 3.223e-05
step:   1040/5000   	 time/step (normalized): 3.191e-05
step:   1050/5000   	 time/step (normalized): 3.75e-05
step:   1060/5000   	 time/step (normalized): 3.308e-05
step:   1070/5000   	 time/step (normalized): 3.217e-05
step:   1080/5000   	 time/step (normalized): 3.568e-05
step:   1090/5000   	 time/step (normalized): 3.391e-05
step:   1100/5000   	 time/step (normalized): 3.378e-05
step:   1110/5000   	 time/step (normalized): 3.525e-05
step:   1120/5000   	 time/step (normalized): 3.766e-05
step:   1130/5000   	 time/step (normalized): 3.31

step:   2430/5000   	 time/step (normalized): 4.097e-05
step:   2440/5000   	 time/step (normalized): 4.044e-05
step:   2450/5000   	 time/step (normalized): 3.819e-05
step:   2460/5000   	 time/step (normalized): 3.721e-05
step:   2470/5000   	 time/step (normalized): 4.649e-05
step:   2480/5000   	 time/step (normalized): 3.508e-05
step:   2490/5000   	 time/step (normalized): 4.742e-05
step:   2500/5000   	 time/step (normalized): 4.297e-05
step:   2510/5000   	 time/step (normalized): 3.481e-05
step:   2520/5000   	 time/step (normalized): 4.403e-05
step:   2530/5000   	 time/step (normalized): 3.933e-05
step:   2540/5000   	 time/step (normalized): 3.918e-05
step:   2550/5000   	 time/step (normalized): 4.326e-05
step:   2560/5000   	 time/step (normalized): 4.879e-05
step:   2570/5000   	 time/step (normalized): 4.131e-05
step:   2580/5000   	 time/step (normalized): 3.676e-05
step:   2590/5000   	 time/step (normalized): 3.454e-05
step:   2600/5000   	 time/step (normalized): 3.

step:   3900/5000   	 time/step (normalized): 3.675e-05
step:   3910/5000   	 time/step (normalized): 3.555e-05
step:   3920/5000   	 time/step (normalized): 4.064e-05
step:   3930/5000   	 time/step (normalized): 4.522e-05
step:   3940/5000   	 time/step (normalized): 3.358e-05
step:   3950/5000   	 time/step (normalized): 3.518e-05
step:   3960/5000   	 time/step (normalized): 3.44e-05
step:   3970/5000   	 time/step (normalized): 3.299e-05
step:   3980/5000   	 time/step (normalized): 3.49e-05
step:   3990/5000   	 time/step (normalized): 3.762e-05
step:   4000/5000   	 time/step (normalized): 3.312e-05
step:   4010/5000   	 time/step (normalized): 3.358e-05
step:   4020/5000   	 time/step (normalized): 3.262e-05
step:   4030/5000   	 time/step (normalized): 3.191e-05
step:   4040/5000   	 time/step (normalized): 3.194e-05
step:   4050/5000   	 time/step (normalized): 3.203e-05
step:   4060/5000   	 time/step (normalized): 3.284e-05
step:   4070/5000   	 time/step (normalized): 3.42

In [39]:
import pickle

with open(model.files['parameters_pkl_file'], 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)